#Setup

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar

!pip3 install atari-py

!python -m atari_py.import_roms ROMS

!pip install "gym[atari]" "gym[accept-rom-license]" atari_py

In [ ]:
!pip install -U "ray[rllib]==1.11.0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import gym
import torch

import numpy as np
from collections import namedtuple
import random, math
import matplotlib.pyplot as plt
from ray.rllib.models import ModelCatalog

from ray.rllib.env.env_context import EnvContext
import time
import os

from gym.spaces import Discrete, Box
from ray import tune
import ray
import ray.rllib.agents.dqn as dqn

In [ ]:
# Visualizer setup
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1


# Code for visualization

In [ ]:
# How to get gym AI visuals in colab
# https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb#scrollTo=T9RpF49oOsZj

from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Test env with random actions



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

game =  'Breakout-ram-v0' 
env =  wrap_env(gym.make(game))

print("game: ", game)
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n

print("States size:", num_states, "Actions size: ",  num_actions)

device:  cuda:0
game:  Breakout-v0
States size: 210 Actions size:  4


In [ ]:
done = False

env.reset()

total_reward = 0 
while not done:
    env.render()
    state, reward, done, _ = env.step(env.action_space.sample()) # take a random action
    total_reward += reward

env.close()
show_video()

In [ ]:
print("Total reward with random actions: {}".format(total_reward))

Total reward with random actions: 2.0


# Use ray_tune for hyperparameter search

In [ ]:
import json
def evaluation_fn(result):
    return result['episode_reward_mean']


def objective_fn(config, checkpoint_dir = "/content/drive/MyDrive/INM707/breakout_v1"):
    start = 0
    if checkpoint_dir:
        with open(os.path.join(checkpoint_dir, "checkpoint")) as f:
            state = json.loads(f.read())
            start = state["step"] + 1


    trainer = dqn.DQNTrainer(config = config) 

    for i in range(start, 100):
      # Perform one iteration of training the policy with DQN
      result = trainer.train()
      intermediate_score = evaluation_fn(result)

      with tune.checkpoint_dir(step=i) as checkpoint_dir:
          path = os.path.join(checkpoint_dir, "checkpoint")
          with open(path, "w") as f:
              f.write(json.dumps({"step": i}))
    
      # Feed the score back back to Tune.
      tune.report(iterations=i, mean_reward=intermediate_score)

##set up config

In [ ]:
ray.shutdown()
ray.init(num_cpus= 2, num_gpus=1)

In [ ]:
config = dqn.DEFAULT_CONFIG.copy()

config["dueling"] = tune.grid_search([True, False])
config["double_q"] = tune.grid_search([True, False])
config["noisy"] = tune.grid_search([True, False])
config["prioritized_replay"] = tune.grid_search([True, False])


#evluation config 
config["evaluation_interval"] =  10
config["evaluation_config"] = {"explore": False} 

config["env"] = 'Breakout-ram-v0' 

config["model"] = {"fcnet_hiddens": [128],
                  "fcnet_activation": 'relu',
                   }

config["num_gpus"] = 1

In [ ]:
analysis = tune.run(
        objective_fn,
        stop={"mean_reward": 5},
        metric="mean_reward",
        mode="max",
        resources_per_trial={'gpu': 1},
        num_samples=3,
        local_dir = "/content/drive/MyDrive/INM707/breakout_v1",
        config=config)



In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

Best hyperparameters found were:  {'num_workers': 0, 'num_envs_per_worker': 1, 'create_env_on_driver': False, 'rollout_fragment_length': 4, 'batch_mode': 'truncate_episodes', 'gamma': 0.99, 'lr': 0.0005, 'train_batch_size': 32, 'model': {'fcnet_hiddens': [128], 'fcnet_activation': 'relu'}, 'optimizer': {}, 'horizon': None, 'soft_horizon': False, 'no_done_at_end': False, 'env': 'Breakout-ram-v0', 'observation_space': None, 'action_space': None, 'env_config': {}, 'remote_worker_envs': False, 'remote_env_batch_wait_ms': 0, 'env_task_fn': None, 'render_env': False, 'record_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'preprocessor_pref': 'deepmind', 'log_level': 'WARN', 'callbacks': <class 'ray.rllib.agents.callbacks.DefaultCallbacks'>, 'ignore_worker_failures': False, 'log_sys_usage': True, 'fake_sampler': False, 'framework': 'tf', 'eager_tracing': False, 'eager_max_retraces': 20, 'explore': True, 'exploration_config': {'type': 'EpsilonGreedy', 'ini

In [ ]:
analysis.best_checkpoint.local_path

'/content/drive/MyDrive/INM707/breakout_v1/objective_fn_2022-04-22_15-37-53/objective_fn_Breakout-ram-v0_2d00f_00033_33_double_q=False,dueling=True,noisy=True,prioritized_replay=True_2022-04-22_17-29-09/checkpoint_000000/'

## Save best params in a file

In [ ]:
#save best config to file 
import json 
#path = '/content/drive/MyDrive/INM707/cartpole_v1/objective_fn_2022-04-21_20-53-06'

path = ''
for val in analysis.best_checkpoint.local_path.split('/')[1:-3]: 
  path = path + '/' + val 

fname = 'best_params.json'
full_path = path + '/' + fname 


f = open(full_path, "w")
json_dict = json.dumps(str(analysis.best_config))
f.write(json_dict)
f.close()


In [ ]:
path = ''
for val in analysis.best_checkpoint.local_path.split('/')[1:-3]: 
  path = path + '/' + val 

df = analysis.dataframe(metric="mean_reward", mode="max")
df = df[['config/noisy', 'config/double_q', 'config/dueling', 'config/prioritized_replay', 'mean_reward']].sort_values( by="mean_reward", ascending=False)#.head()

fname = "analysis_df.csv"
full_path =  path + '/' + fname

df.to_csv(full_path)
print("Saved to: ", full_path)

Saved to:  /content/drive/MyDrive/INM707/breakout_v1/objective_fn_2022-04-22_15-37-53/analysis_df.csv


In [ ]:
df[['config/noisy', 'config/double_q', 'config/dueling', 'config/prioritized_replay', 'mean_reward']].sort_values( by="mean_reward", ascending=False).head()

,config/noisy,config/double_q,config/dueling,config/prioritized_replay,mean_reward
33,True,False,True,True,5.000000
13,False,False,True,False,3.500000
43,True,False,False,False,3.000000
35,True,False,False,True,3.000000
24,True,True,True,False,2.333333
